In [1]:
import requests
import pandas as pd
import time
import html.parser as htmlparser
import html
import re

In [853]:
#Keys
API_KEY = "***Add Your Key Here"
CHANNEL_ID = "UCEv62iwobOfIHwKUw_RU9mQ"

In [854]:
def deEmojify(text):
    "function to remove emojis from text"
    regrex_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002500-\U00002BEF"  # chinese char
        u"\U00002702-\U000027B0"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642" 
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"  # dingbats
        u"\u3030"
                      "]+", re.UNICODE)
    return regrex_pattern.sub(r'',text)

In [855]:
def get_video_details(video_id):

    #collecting view, like, dislike, comment counts
    url_video_stats = "https://www.googleapis.com/youtube/v3/videos?id="+video_id+"&part=statistics,snippet,contentDetails&key="+API_KEY
    response_video_stats = requests.get(url_video_stats).json()

    view_count = response_video_stats['items'][0]['statistics']['viewCount']
    like_count = response_video_stats['items'][0]['statistics']['likeCount']
    dislike_count = response_video_stats['items'][0]['statistics']['dislikeCount']
    comment_count = response_video_stats['items'][0]['statistics']['commentCount']
    video_description_full = response_video_stats['items'][0]['snippet']['description']
    video_tags = response_video_stats['items'][0]['snippet'].get('tags')
    video_categoryID = response_video_stats['items'][0]['snippet']['categoryId']
    video_definition = response_video_stats['items'][0]['contentDetails']['definition']
    video_duration = response_video_stats['items'][0]['contentDetails']['duration']
    video_caption = response_video_stats['items'][0]['contentDetails']['caption']
    video_licensedContent = response_video_stats['items'][0]['contentDetails']['licensedContent']


    return view_count, like_count, dislike_count, comment_count, video_description_full, video_tags, video_categoryID, video_definition, video_duration, video_caption, video_licensedContent

In [856]:
def get_channel_details(CHANNEL_ID):

    #collecting view, like, dislike, comment counts
    url_channel_stats = "https://www.googleapis.com/youtube/v3/channels?id="+CHANNEL_ID+"&part=statistics,snippet,topicDetails,status,brandingSettings&key="+API_KEY
    response_video_stats = requests.get(url_channel_stats).json()

    channel_title = response_video_stats['items'][0]['snippet']['title']
    channel_description = response_video_stats['items'][0]['snippet']['description']
    channel_customUrl = response_video_stats['items'][0]['snippet']['customUrl']
    channel_publishedAt = response_video_stats['items'][0]['snippet']['publishedAt']
    channel_country = response_video_stats['items'][0]['snippet']['country']
    
    channel_subscriberCount = response_video_stats['items'][0]['statistics']['subscriberCount']
    channel_hiddenSubscriberCount = response_video_stats['items'][0]['statistics']['hiddenSubscriberCount']
    channel_videoCount = response_video_stats['items'][0]['statistics']['videoCount']
    channel_viewCount = response_video_stats['items'][0]['statistics']['viewCount']
    
    channel_topicCategories = response_video_stats['items'][0]['topicDetails']['topicCategories']
    
    channel_privacyStatus = response_video_stats['items'][0]['status']['privacyStatus']
    channel_longUploadsStatus = response_video_stats['items'][0]['status']['longUploadsStatus']
    channel_madeForKids = response_video_stats['items'][0]['status']['madeForKids']
    
    channel_keywords = response_video_stats['items'][0]['brandingSettings']['channel']['keywords']
    
    return channel_title, channel_description, channel_customUrl, channel_publishedAt, channel_country,channel_subscriberCount, channel_hiddenSubscriberCount, channel_videoCount, channel_viewCount, channel_topicCategories,channel_privacyStatus, channel_longUploadsStatus, channel_madeForKids, channel_keywords

In [857]:
def get_videos(df):
    pageToken = ""
    i = 0
    
    channel_title, channel_description, channel_customUrl, channel_publishedAt, channel_country, channel_subscriberCount, channel_hiddenSubscriberCount, channel_videoCount, channel_viewCount, channel_topicCategories, channel_privacyStatus, channel_longUploadsStatus, channel_madeForKids, channel_keywords= get_channel_details(CHANNEL_ID)
    
    while (i < 101):
        url = "https://www.googleapis.com/youtube/v3/search?key="+API_KEY+"&channelId="+CHANNEL_ID+"&part=snippet,id&order=date&maxResults=10000&"+pageToken
        response = requests.get(url).json()
        time.sleep(1) #give it a second before starting the for loop
        for video in response['items']:
            if video['id']['kind'] == "youtube#video":
                video_id = video['id']['videoId']
                video_title = video['snippet']['title']
                video_description_short = video['snippet']['description']
                video_thumbnail = video['snippet']['thumbnails']['default']
                video_title = html.unescape(video_title)
                video_title = deEmojify(video_title)
                upload_date = video['snippet']['publishedAt']
                upload_date = str(upload_date).split("T")[0]
                view_count, like_count, dislike_count, comment_count , video_description_full, video_tags, video_categoryID, video_definition, video_duration, video_caption, video_licensedContent= get_video_details(video_id)
                i = i+1
                df = df.append({'video_id':video_id,'video_title':video_title, 
                                'video_description_short':video_description_short, 'video_thumbnail':video_thumbnail,
                                "upload_date":upload_date,"view_count":view_count,
                                "like_count":like_count,"dislike_count":dislike_count,
                                "comment_count":comment_count,"video_description_full":video_description_full,
                               "video_tags":video_tags,"video_categoryID":video_categoryID,
                               "video_definition":video_definition,"video_duration":video_duration,
                                "video_caption":video_caption,
                               "video_licensedContent":video_licensedContent,'channel_title':channel_title,
                               'channel_description':channel_description,'channel_customUrl':channel_customUrl,                                
                                'channel_publishedAt':channel_publishedAt,'channel_country':channel_country,                                
                                'channel_subscriberCount':channel_subscriberCount,
                                'channel_hiddenSubscriberCount':channel_hiddenSubscriberCount,'channel_videoCount':channel_videoCount,                                
                                'channel_viewCount':channel_viewCount,'channel_topicCategories':channel_topicCategories,                                
                                'channel_privacyStatus':channel_privacyStatus,'channel_longUploadsStatus':channel_longUploadsStatus,                                
                                'channel_madeForKids':channel_madeForKids,'channel_keywords':channel_keywords},ignore_index=True)
                
        try:
            if response['nextPageToken'] != None: #if none, it means it reached the last page and break out of it
                pageToken = "pageToken=" + response['nextPageToken']

        except:
            break


    return df

In [858]:
#main

#build our dataframe

df2 = pd.DataFrame(columns=["video_id","video_title","video_description_short","video_description_full","video_thumbnail",
                            "upload_date","view_count","like_count","dislike_count","comment_count","video_tags",
                            "video_categoryID","video_definition","video_duration","video_caption","video_licensedContent","channel_title",
                            "channel_description","channel_customUrl","channel_publishedAt","channel_country",
                            "channel_subscriberCount","channel_hiddenSubscriberCount",
                            "channel_viewCount","channel_topicCategories","channel_privacyStatus",                            
                            "channel_longUploadsStatus","channel_madeForKids","channel_keywords"])

df2 = get_videos(df2)

KeyError: 'items'

In [ ]:
API_Clean = df2.astype({'video_id':'string',
            'video_title':'string',
            'video_description_short':'string',
            'video_thumbnail':'string',
           'upload_date':'string',
           'view_count':'int32',
           'like_count':'int32',
           'dislike_count':'int32'})


In [ ]:
API_Clean.to_excel(r'C:\Users\a_sky\OneDrive\Documents\UChicago\21 Fall\Data Engineering Platforms\Assignment1/2.33.xlsx', index = False)